In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
%matplotlib inline

# III/ Comparaison de l'erreur de prédiction de différentes méthodes statistiques

Jusqu'à maintenant, nous avons démontré que les stratégies d'investissements obtenues à partir de la résolution du problème de Markovitz (équivalent à la résolution d'un problème des moindres carrés => démonstration dans le fichier LaTeX) ne sont pas très convaincantes. Nous allons donc tester plusieurs autres méthodes statistiques de prédiction des rendements futurs et allons mettre en place une métrique commune qui mesure l'erreur de prédiction obtenue. 



Ici on crée l'interface qui nous permet de choisir le nombre d'actifs considéré, le nombre de dates, la vraie loi des rendements

In [101]:
nb_actifs = 20
nb_dates = 256 * 5

idx = pd.Index(np.arange(nb_actifs))
idx_date = pd.Index(np.arange(nb_dates))

esp_rdt = 0.05
vol_rdt = 0.2
correl = 0.7
seuil_pb_marko = 0.05
mu = pd.Series(esp_rdt,index=idx)
vols = pd.Series(vol_rdt,index=idx)
allocation_optimale_théorique = pd.Series(1, index = idx_date)

#Matrice var-covar théorique selon la vraie loi des rendements
covar = np.diag(vols) @ pd.DataFrame(correl * np.ones((nb_actifs,nb_actifs)) + (1-correl) * np.eye(nb_actifs),index=idx,columns=idx) @ np.diag(vols)

In [12]:
#On simuler un tirage de rendements suivant une loi normale d'espérence Mu et de Var la matrice Covar
A = np.linalg.cholesky(covar)
rdts_observes = mu/256 + pd.DataFrame(np.random.randn(nb_dates,nb_actifs)) @ A.T / 16

Ici on calcule les estimateurs d'espérance et de variance empiriques de la série des observations

In [34]:
mu_estime = rdts_observes.mean() * 256
covar_estimee = rdts_observes.cov() * 256

#Suivant la formule de la résolution du problème, on calcule l'allocation optimale "in sample"

covar_inv = np.linalg.inv(covar_estimee)
df_covar_inv = pd.DataFrame(covar_inv,index=idx,columns=idx)
lambda1 = seuil_pb_marko/(mu_estime @ df_covar_inv @ mu_estime)
alloc_optimale_in_sample = lambda1 * (df_covar_inv @ mu_estime)

A priori, nous pouvons nous ramener à la résolution d'un problème des moindres carrés. Donc nous allons effectuer la régression de la vraie allocation optimale sur les rendements observés. Le Beta que l'on obtient devrait être le plus proche possible de l'allocation optimale théorique. 

In [86]:
print(rdts_observes.T)

        0         1         2         3         4         5         6     \
0   0.035819  0.004819  0.015750 -0.000811  0.000088  0.009532  0.008290   
1   0.023104 -0.008425  0.013663  0.004288 -0.007336  0.005909  0.008291   
2   0.024473  0.013284  0.026510 -0.004831 -0.001759 -0.005597  0.002767   
3   0.023098  0.006991  0.028677 -0.006470  0.013574  0.006072  0.020630   
4   0.025458  0.006909  0.015991  0.015400  0.003164  0.008677  0.000235   
5   0.029359  0.021504  0.003523  0.001684 -0.007916  0.005239 -0.002551   
6   0.021993  0.010473  0.012210  0.002906  0.008159 -0.003739  0.005110   
7   0.026686  0.016442  0.016223  0.008265  0.006621  0.004804  0.004162   
8   0.028976  0.006106  0.019859  0.002693  0.008161  0.007217  0.007790   
9   0.023009  0.002292  0.031546  0.000395  0.006052  0.003220  0.011795   
10  0.011865 -0.000325  0.017819  0.013033 -0.011886  0.015763  0.006431   
11  0.021652 -0.004697  0.028186 -0.011423  0.003507  0.011370 -0.000690   
12  0.038732

In [103]:
regression = LinearRegression()

# Entraîner le modèle sur les données
regression.fit(rdts_observes, allocation_optimale_théorique)

# Obtenir les coefficients de la régression
coefficients = regression.coef_
intercept = regression.intercept_

In [104]:
print(coefficients)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
